In [0]:
CREATE TABLE IF NOT EXISTS IDENTIFIER(:catalog || '.' || :schema || '.references') (
  path STRING,
  agreements ARRAY<STRING>,
  references ARRAY<STRUCT<section: STRING, document: STRING>>
)
USING DELTA;

MERGE INTO IDENTIFIER(:catalog || '.' || :schema || '.references') AS target
USING (
  WITH ref_results AS (
  SELECT
  path,
  from_json(
    AI_QUERY(
    "databricks-claude-sonnet-4-5",
    CONCAT(
      'You are a contract expert. 
      
      1. Look over the attached document and list any of the following agreements with the agreement type. This should be a specfic agreement with a number attached to it. e.g. Master Agreement 1239-12900.      
      - Master Agreement
      - Framework Agreement
      - Consulting Agreement (CSA)
      - NDA (Non Disclosure Agreement) / Confidentiality
      - Purchase Order Terms and Conditions
      - Mutually Agreed to Terms and Conditions (MTC)
      - Contract
      - Master Work Agreement (MWA)
      - Sales Contract
      - Engineering Procurement Construction (EPC)
      - Engineering Procurement Construction Management (EPCM)
      - Construction Agreement
      - Site Services Agreement
      - Staffing Agreement
      - Sales/Catering Contract
      - Recruitment Agreement
      - Administration Services Agreement
      - Services Agreement
      - License Agreement
      - Supply Agreement
      - Order Form
      - Purchase Agreement
      - General Terms and Conditions
      
      2. Look at each section of the contract and list any section that references other documents or contractual numbers. This should be a real document with some form of reference like a number, not a hypothetical general document. e.g. Economic Disclosure Statement: Ownership Interest Declaration (EDS-7: 3/2015)
      - Amendments
      - Rate sheets
      - Schedules
      - Exhibits
      - Addendums
      - Statement of Work (SOW)
      - Termination
      - Forms of Undertaking (FOU)
      - Commitment Letter
      - Change Order
      
      Output the results in JSON format: {
        "agreements": ["agreement 1 + number", "agreement 2 + number"],
        "references": [
          {"section": "section_name_1", "document": "document1"},
          {"section": "section_name_2", "document": "document2"}
        ]
      }
      
      ## Document ##',
      'Vendor Name:', vendor_name, '\n',
      'File Name:', file_name, '\n',
      'Text:', truncated, '\n'
    ),
    responseFormat => '{
      "type": "json_schema", 
      "json_schema": {
        "name": "result", 
        "schema": {
          "type": "object", 
          "properties": {
            "agreements": {
              "type": "array", 
              "items": {"type": "string"}
            },
            "references": {
              "type": "array",
              "items": {
                "type": "object", 
                "properties": {
                  "section": {"type": "string"},
                  "document": {"type": "string"}
                }
              }
            }
          }
        },
        "strict": true
      }
    }'
  ),
  'STRUCT<agreements:ARRAY<STRING>, references:ARRAY<STRUCT<section:STRING, document:STRING>>>'
  ) as result
FROM IDENTIFIER(:catalog || '.' || :schema || '.flat')
)
SELECT 
  path,
  result.agreements as agreements,
  result.references as references
FROM ref_results
 ) AS source
ON target.path = source.path
WHEN NOT MATCHED THEN INSERT *;